# Homework 4 - Spark

In this homework, we are practicing Apache Spark.

You are required to turn in this notebook as BDM\_HW4\_Spark\_**NetId**.ipynb. You will be asked to complete each task using Apache Spark. Output can be printed in the notebook.

## Task 1 (5 points)

You are asked to implement Homework 3 using Spark. The description is provided below for your convenience.

You are asked to implement the Social Triangle example discussed in class. In particular, given the email dataset, please list all "reciprocal" relationships in the company. Recall that:

If A emails B and B emails A, then A and B is *reciprocal*.

If A emails B but B doesn’t email A, then A and B is *directed*.

**Dataset:** We will use a subset of the open [Enron Email Dataset](https://www.cs.cmu.edu/~./enron/ "Enron Email Dataset"), which contains approximately 10,000 simplified email headers from the Enron Corporation. You can download this dataset from NYU Classes as **enron_mails_small.csv**. The file contains 3 columns *Date*, *From*, and *To*. Their description is as follows:

|Column name|Description|
|--|--|
|Date |The date and time of the email, in the format YYYY-MM-DD hh-mm-ss, <br />e.g. "1998-10-30 07:43:00" |
|From |The sender email address, <br />e.g. "mark.taylor@enron.com" |
|To | A list of recipients' email addresses separated by semicolons ';', <br />e.g. "jennifer.fraser@enron.com;jeffrey.hodge@enron.com" |

Note that, we only care about users employed by Enron, or only relationships having email addresses that end with *'@enron.com'*.

The expected output is also provided below. For each reciprocal relationship, please output a tuple consisting of two strings. The first one is always **'reciprocal'**. And the second one is a string showing the name of the two person in the following format: **'Jane Doe : John Doe'**. The names should be presented in the lexical order, i.e. there will not be a 'John Doe : Jane Doe' since 'Jane' is ordered before 'John.

Though the dataset only contains email addresses, not actual names, we're assuming that the email aliases were created based on their name. For example:

|Email Address|Converted Name|
|--|--|
|mark.taylor@enron.com|Mark Taylor|
|alan.aronowitz@enron.com|Alan Aronowitz|
|marc.r.cutler@enron.com|Marc R Cutler|
|hugh@enron.com|Hugh|

Please fill the code block with a series of MapReduce jobs using your own mapper and reducer functions. Be sure to include the naming convention logic into one of your mappers and/or reducers.

In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [78]:
mailrecord = sc.textFile('enron_mails_small.csv').cache()
mailrecord.first()

'Date,From,To'

In [240]:
def getinfo(pid,rows):
    if pid ==0:
        next(rows)
    import csv
    row = csv.reader(rows)
    for info in row:
        fromaddress = info[1].split('@')[1]
        if fromaddress=='enron.com':
            for to in info[2].split(';'):
                if to.split('@')[1]=='enron.com':
                    yield ((info[1].split('@')[0],to.split('@')[0].split(';')[0]),1)
                    

In [253]:
recipocal = mailrecord.mapPartitionsWithIndex(getinfo).reduceByKey(lambda x,y: 1)\
            .map(lambda x: ((sorted(x[0])[0],sorted(x[0])[1]), 1))\
            .reduceByKey(lambda x,y: x+y)\
.filter(lambda x: x[1] % 2 == 0)\
.map(lambda x:('recipropcal',x[0][0].split('.')[0].capitalize()+' '+x[0][0].split('.')[1].capitalize()+' : ' + x[0][1].split('.')[0].capitalize()+' '+x[0][1].split('.')[1].capitalize()))

recipocal.collect()

[('recipropcal', 'Carol Clair : Mark Taylor'),
 ('recipropcal', 'Elizabeth Sager : Mark Taylor'),
 ('recipropcal', 'Richard Sanders : Sara Shackleton'),
 ('recipropcal', 'Sara Shackleton : Tana Jones'),
 ('recipropcal', 'Brenda Whitehead : Elizabeth Sager'),
 ('recipropcal', 'Gerald Nemec : Susan Scott'),
 ('recipropcal', 'Greg Whalley : Richard Sanders'),
 ('recipropcal', 'Mark Haedicke : Richard Sanders'),
 ('recipropcal', 'Janette Elbertson : Richard Sanders'),
 ('recipropcal', 'Stinson Gibner : Vince Kaminski'),
 ('recipropcal', 'Shirley Crenshaw : Vince Kaminski'),
 ('recipropcal', 'Drew Fossum : Susan Scott'),
 ('recipropcal', 'Michelle Cash : Twanda Sweet'),
 ('recipropcal', 'Grant Masson : Vince Kaminski'),
 ('recipropcal', 'Fletcher Sturm : Sally Beck'),
 ('recipropcal', 'Fletcher Sturm : Greg Whalley'),
 ('recipropcal', 'Liz Taylor : Mark Haedicke'),
 ('recipropcal', 'Carol Clair : Debra Perlingiere'),
 ('recipropcal', 'Debra Perlingiere : Kevin Ruscitti'),
 ('recipropcal', '

## Task 2 (5 points)

You are asked to implement Task 2 of Lab 5. The description is provided below for your convenience.

We’ll be using two NYC open data sets: the SAT Results and the NYC High School Directory data sets. Both can be downloaded from the links below, or from online class resources.

**Dataset**: *Please note that each school is uniquely identified by an DBN code, which should be found on both data sets.*

**SAT_Results.csv**
Source: https://nycopendata.socrata.com/Education/SAT-Results/f9bf-2cp4  
Description: “The most recent school level results for New York City on the SAT. Results are available at the school level for the graduating seniors of 2012.”

**DOE_High_School_Directory_2014-2015.csv**
Source: https://data.cityofnewyork.us/Education/DOE-High-School-Directory-2014-2015/n3p6-zve2  
Description: “Directory of NYC High Schools.”

We would like to know how the Math scores vary across bus lines or subway lines serving the schools. Your task is to compute the average Math scores of all schools along each bus line and subway line. You can find the bus and subway lines serving each school in the High School Dictionary as bus and subway columns.

The expected results are two lists:
1. A list of key/value pairs: with bus line as keys, and the average Math scores as values.
2. A list of key/value pairs: with subway line as keys, and the average Math scores as values.

The top ten lines with highest score are shown below.

In [456]:
sat = sc.textFile('SAT_Results.csv').cache()
doe = sc.textFile('DOE_High_School_Directory_2014-2015.csv').cache()

In [481]:
def extractScores(partitionId,rows):
    if partitionId ==0:
        next(rows)
    import csv
    reader = csv.reader(rows)
    for fields in reader:
        if fields[2]!='s':
            yield(fields[0],(int(fields[4]),int(fields[2])))

def extractSchools(pid, rows):
    if pid==0:
        next(rows)
    import csv
    reader = csv.reader(rows)
    for fields in reader:
        if len(fields) ==58 and fields[17].isdigit():
            if fields[10]!='N/A':
                for bus in fields[10].split(','):
                    yield (fields[0], bus.replace(' ',''))

In [483]:
BusSchool = doe.mapPartitionsWithIndex(extractSchools)
BusSchool.take(2)

[('01M292', 'B39'), ('01M292', 'M14A')]

In [466]:
SatScore = sat.mapPartitionsWithIndex(extractScores)

In [509]:
BusSchool.join(SatScore).values()\
.mapValues(lambda x:(x[0]*x[1],x[1]))\
.reduceByKey(lambda x,y :(x[0]+y[0],x[1]+y[1]))\
.mapValues(lambda x:round(x[0]/x[1]))\
.collect()

[('M14A', 476),
 ('M21', 486),
 ('M103', 441),
 ('M3', 433),
 ('M34A-SBS', 438),
 ('M50', 439),
 ('M57', 486),
 ('M4', 443),
 ('M72', 523),
 ('M100', 428),
 ('Bx41-SBS', 390),
 ('Bx32', 398),
 ('Bx36', 373),
 ('Bx3', 572),
 ('Bx38', 379),
 ('Bx31', 417),
 ('Bx27', 371),
 ('B69', 549),
 ('B65', 538),
 ('B24', 432),
 ('B32', 409),
 ('B44', 465),
 ('B48', 412),
 ('Q54', 408),
 ('Q59', 419),
 ('B15', 383),
 ('B14', 391),
 ('B83', 376),
 ('Q56', 434),
 ('B1', 461),
 ('B64', 455),
 ('B11', 504),
 ('Q66', 462),
 ('Q41', 441),
 ('Q111', 446),
 ('Q113', 439),
 ('Q17', 496),
 ('Q20A', 505),
 ('Q3', 456),
 ('Q76', 488),
 ('Q8', 443),
 ('Q69', 444),
 ('S56', 477),
 ('S51', 422),
 ('S74', 486),
 ('M106', 470),
 ('Bx20', 455),
 ('Q53', 438),
 ('Q27', 481),
 ('Q30', 508),
 ('Q1', 413),
 ('Q43', 433),
 ('Q77', 413),
 ('S54', 474),
 ('B100', 475),
 ('Q15A', 438),
 ('Q28', 493),
 ('Q22', 435),
 ('S59', 474),
 ('Q34', 441),
 ('Q48', 441),
 ('Q10', 404),
 ('M9', 539),
 ('M15-SBS', 453),
 ('M8', 467),
 ('M

In [537]:
def extractSchools(pid, rows):
    if pid==0:
        next(rows)
    import csv
    reader = csv.reader(rows)
    for fields in reader:
        if len(fields) ==58 and fields[17].isdigit():
            if fields[11]!='N/A':
                for subs in fields[11].replace(' to ',';').replace(' ','').split(';')[::2]:
                    for sub in subs.split(','):
                        yield (fields[0], sub)

In [538]:
SubSchool = doe.mapPartitionsWithIndex(extractSchools)
SubSchool.join(SatScore).values()\
.mapValues(lambda x:(x[0]*x[1],x[1]))\
.reduceByKey(lambda x,y :(x[0]+y[0],x[1]+y[1]))\
.mapValues(lambda x:round(x[0]/x[1]))\
.collect()

[('J', 439),
 ('L', 426),
 ('4', 495),
 ('N', 494),
 ('F', 446),
 ('M', 454),
 ('6', 433),
 ('B', 492),
 ('3', 513),
 ('A', 510),
 ('Q', 482),
 ('7', 457),
 ('C', 510),
 ('1', 500),
 ('R', 509),
 ('S', 428),
 ('SIR', 499),
 ('Z', 438),
 ('D', 502),
 ('E', 501),
 ('2', 488),
 ('5', 461),
 ('G', 503)]